In [1]:
import os
import shutil
from pathlib import Path
import numpy as np
import random

np.random.seed(0) 
random.seed(0)


In [2]:
import torch
import yaml
import shutil
import optuna
from pathlib import Path
from ultralytics import YOLO
import cv2
import numpy as np
from collections import defaultdict

class AdvancedYOLOTrainer:
    def __init__(self, base_model):
        self.base_model = base_model
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
    def create_advanced_config(self):
        config = {
            # Hiperparâmetros otimizados para contexto cirúrgico
            'lr0': 0.001,
            'lrf': 0.01,
            'momentum': 0.98,
            'weight_decay': 0.0005,
            'warmup_epochs': 10,
            'warmup_momentum': 0.8,
            'warmup_bias_lr': 0.1,
            
            # Configurações de loss otimizadas
            'box': 7.5,
            'cls': 0.5,
            'dfl': 1.5,
            
            # Configurações de treino
            'epochs': 10, # 150
            'batch': 2,
            'imgsz': 640,
            'patience': 50,
            'save_period': 10,
            
            # Augmentações específicas para ambiente cirúrgico
            'hsv_h': 0.015,
            'hsv_s': 0.7,
            'hsv_v': 0.4,
            'degrees': 15.0,
            'translate': 0.1,
            'scale': 0.5,
            'shear': 5.0,
            'perspective': 0.0001,
            'flipud': 0.0,
            'fliplr': 0.5,
            'mosaic': 1.0,
            'mixup': 0.15,
            'copy_paste': 0.3,
            
            # Regularização avançada
            'dropout': 0.2,
            'val': True,
            'plots': True,
            'verbose': True
        }
        return config

    def setup_multiphase_training(self):
        phases = {
            'phase1': {
                'description': 'COCO pre-training com backbone congelado',
                'epochs': 10, # 50
                'freeze': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                'lr0': 0.01,
                'momentum': 0.937
            },
            'phase2': {
                'description': 'Fine-tuning completo em dados cirúrgicos',
                'epochs': 10, # 100
                'freeze': None,
                'lr0': 0.001,
                'momentum': 0.98
            },
            'phase3': {
                'description': 'Adaptação final com regularização forte',
                'epochs': 10, # 50
                'freeze': None,
                'lr0': 0.0001,
                'momentum': 0.99,
                'weight_decay': 0.001
            }
        }
        return phases

    def optimize_hyperparameters(self, n_trials=50):
        """Otimização automática de hiperparâmetros usando Optuna"""
        def objective(trial):
            lr0 = trial.suggest_float('lr0', 1e-5, 1e-1, log=True)
            momentum = trial.suggest_float('momentum', 0.8, 0.99)
            weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-2, log=True)
            box_loss = trial.suggest_float('box', 5.0, 10.0)
            cls_loss = trial.suggest_float('cls', 0.3, 1.0)
            
            model = YOLO(self.base_model)
            results = model.train(
                data=Path("yolo_dataset/data.yaml").resolve(),
                epochs=2,  
                lr0=lr0,
                momentum=momentum,
                weight_decay=weight_decay,
                box=box_loss,
                cls=cls_loss,
                verbose=False,
                plots=False
            )
            
            return results.results_dict.get('metrics/mAP50(B)', 0.0)
        
        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=n_trials)
        
        return study.best_params

    def train_with_advanced_features(self, data_path, use_optimization=True):
        
        if use_optimization:
            best_params = self.optimize_hyperparameters(n_trials=20)
        else:
            best_params = {}
        
        config = self.create_advanced_config()
        config.update(best_params)
        
        phases = self.setup_multiphase_training()
        
        best_model = None
        best_map = 0.0
        
        for phase_name, phase_config in phases.items():
            
            if phase_name == 'phase1':
                model = YOLO(self.base_model)
            else:
                model = YOLO('runs/detect/train/weights/best.pt')
            
            current_config = config.copy()
            current_config.update({
                'epochs': phase_config['epochs'],
                'lr0': phase_config.get('lr0', config['lr0']),
                'momentum': phase_config.get('momentum', config['momentum']),
                'freeze': phase_config.get('freeze'),
                'name': f'train_{phase_name}'
            })
            
            results = model.train(
                data=data_path,
                **current_config
            )
            
            current_map = results.results_dict.get('metrics/mAP50(B)', 0.0)
            if current_map > best_map:
                best_map = current_map
                best_model = f'runs/detect/train_{phase_name}/weights/best.pt'
            
        return best_model, best_map

class MOTChallengeKeypointTracker:
    """Classe para tracking de keypoints no formato MOTChallenge modificado"""
    
    def __init__(self, model_path):
        self.model = YOLO(model_path)
        self.tracker_id = 0
        self.active_tracks = {}
        
    def extract_keypoints_from_detection(self, detection):
        """Extrai keypoints de uma detecção YOLO"""
        # Assumindo que o modelo detecta instrumentos e seus keypoints
        # Esta função deve ser adaptada baseada na arquitetura específica do modelo
        bbox = detection['bbox']  # [x1, y1, x2, y2]
        class_id = detection['class_id']
        confidence = detection['confidence']
        
        # Keypoints específicos por classe de instrumento
        keypoint_configs = {
            'needle': ['tip', 'base'],
            'clamp': ['tip1', 'tip2', 'hinge'],
            'curved_scissor': ['tip1', 'tip2', 'hinge'],
            'straight_scissor': ['tip1', 'tip2', 'hinge']
        }
        
        # Simular extração de keypoints (deve ser substituído por lógica real)
        keypoints = []
        class_name = ['needle', 'clamp', 'curved_scissor', 'straight_scissor'][class_id]
        
        for i, kp_name in enumerate(keypoint_configs[class_name]):
            # Coordenadas simuladas baseadas na bbox
            x = bbox[0] + (bbox[2] - bbox[0]) * (i + 1) / (len(keypoint_configs[class_name]) + 1)
            y = bbox[1] + (bbox[3] - bbox[1]) * 0.5
            kp_confidence = confidence * 0.9  # Confiança ligeiramente menor para keypoints
            
            keypoints.extend([x, y, kp_confidence])
        
        return keypoints
    
    def track_frame(self, frame, frame_id):
        """Processa um frame e retorna tracking no formato MOTChallenge modificado"""
        results = self.model(frame)
        mot_lines = []
        
        for detection in results[0].boxes.data:
            # Converter detecção YOLO para formato interno
            det_dict = {
                'bbox': detection[:4].cpu().numpy(),
                'confidence': detection[4].cpu().numpy(),
                'class_id': int(detection[5].cpu().numpy())
            }
            
            # Extrair keypoints
            keypoints = self.extract_keypoints_from_detection(det_dict)
            
            # Atribuir ID de tracking (simplificado - usar tracker mais sofisticado em produção)
            track_id = self.assign_track_id(det_dict, frame_id)
            
            # Criar linha no formato MOTChallenge modificado
            mot_line = f"{frame_id},{track_id}"
            for kp in keypoints:
                mot_line += f",{kp:.2f}"
            
            mot_lines.append(mot_line)
        
        return mot_lines
    
    def assign_track_id(self, detection, frame_id):
        """Atribui ID de tracking - implementação simplificada"""
        self.tracker_id += 1
        return self.tracker_id
    
    def process_video(self, video_path, output_path):
        """Processa vídeo completo e gera arquivo de tracking"""
        cap = cv2.VideoCapture(str(video_path))
        frame_id = 1
        all_tracks = []
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_tracks = self.track_frame(frame, frame_id)
            all_tracks.extend(frame_tracks)
            frame_id += 1
        
        cap.release()
        
        # Salvar no formato MOTChallenge modificado
        with open(output_path, 'w') as f:
            for track in all_tracks:
                f.write(track + '\n')
        
        return output_path

In [3]:
def main():
    # Treino do modelo base
    trainer = AdvancedYOLOTrainer("yolo11n.pt")
    
    root = Path("../Surgical-Dataset")
    img_dir = root / "Images"
    obj_label_dir = root / "Labels/label object names"
    
    yolo_dir = Path("yolo_dataset")
    (yolo_dir / "images/train").mkdir(parents=True, exist_ok=True)
    (yolo_dir / "labels/train").mkdir(parents=True, exist_ok=True)
    
    # Preparação dos dados
    for label_file in obj_label_dir.glob("*.txt"):
        img_id = label_file.stem
        img_file = img_dir / f"{img_id}.jpg"
        if img_file.exists():
            shutil.copy(img_file, yolo_dir / "images/train" / f"{img_id}.jpg")
            shutil.copy(label_file, yolo_dir / "labels/train" / f"{img_id}.txt")
    
    label_map = {
        0: "bisturi",
        1: "pinca", 
        2: "tesouracurva",
        3: "tesourareta",
    }
    
    label_remap = {
        "bisturi": "needle",
        "pinca": "clamp", 
        "tesouracurva": "curved_scissor",
        "tesourareta": "straight_scissor"
    }
    
    # Configuração do dataset
    data_config = {
        'path': str(yolo_dir.absolute()),
        'train': 'images/train',
        'val': 'images/train',
        'nc': 4,
        'names': [label_remap.get(label_map[i], label_map[i]) for i in sorted(label_map.keys())]
    }
    
    with open(yolo_dir / "data.yaml", "w") as f:
        yaml.dump(data_config, f)
    
    data_path = Path("yolo_dataset/data.yaml").resolve()
    best_model_path, best_map = trainer.train_with_advanced_features(
        data_path=data_path,
        use_optimization=True
    )
    
    print(f"Treino concluído!")
    print(f"Melhor mAP@0.5: {best_map:.4f}")
    print(f"Melhor modelo salvo em: {best_model_path}")
    
    if best_model_path:
        print("\nIniciando tracking de keypoints...")
        
        # Inicializar tracker
        tracker = MOTChallengeKeypointTracker(best_model_path)
        
        # Exemplo de processamento de vídeo
        # video_path = "path/to/surgical_video.mp4"
        # output_path = "tracking_results_mot_modified.txt"
        # tracker.process_video(video_path, output_path)
        
        print("Framework de tracking configurado!")
        print("Formato de saída: MOTChallenge modificado com keypoints")
        print("Estrutura: <frame>,<id>,<x1>,<y1>,<c1>,<x2>,<y2>,<c2>,...,<xK>,<yK>,<cK>")

if __name__ == "__main__":
    main()

[I 2025-05-27 13:28:16,847] A new study created in memory with name: no-name-4e2fc8f7-c4d8-4787-939c-68bc6b5f92c9


New https://pypi.org/project/ultralytics/8.3.145 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.139 🚀 Python-3.10.16 torch-2.7.0+cu128 CUDA:0 (NVIDIA RTX A6000, 48548MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=8.70800181260047, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.6714094007883091, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/mounts/grupo1/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=2, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=1.275439823852573e-05, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.9165008507146155, mosaic=1.0, m

train: Scanning /mounts/grupo1/yolo_dataset/labels/train.cache... 3009 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3009/3009 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1951.3±971.6 MB/s, size: 77.6 KB)


val: Scanning /mounts/grupo1/yolo_dataset/labels/train.cache... 3009 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3009/3009 [00:00<?, ?it/s]


...

      10/10     0.443G     0.9128      1.824      1.863          1        640: 100%|██████████| 1505/1505 [02:40<00:00,  9.38it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 753/753 [00:35<00:00, 21.16it/s]

                   all       3009       4207      0.613      0.718      0.729      0.453



10 epochs completed in 0.544 hours.
Optimizer stripped from runs/detect/train_phase33/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train_phase33/weights/best.pt, 5.5MB

Validating runs/detect/train_phase33/weights/best.pt...
Ultralytics 8.3.139 🚀 Python-3.10.16 torch-2.7.0+cu128 CUDA:0 (NVIDIA RTX A6000, 48548MiB)
YOLO11n summary (fused): 100 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 753/753 [00:23<00:00, 32.63it/s]


                   all       3009       4207      0.646      0.717      0.735      0.454
                needle       1038       1038      0.639      0.734      0.775      0.461
                 clamp       1056       1056      0.665      0.638      0.697      0.411
        curved_scissor        971        971      0.575      0.765        0.7       0.44
      straight_scissor       1142       1142      0.703      0.731      0.768      0.503
Speed: 0.3ms preprocess, 2.1ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/train_phase33
Treino concluído!
Melhor mAP@0.5: 0.8642
Melhor modelo salvo em: runs/detect/train_phase1/weights/best.pt

Iniciando tracking de keypoints...
Framework de tracking configurado!
Formato de saída: MOTChallenge modificado com keypoints
Estrutura: <frame>,<id>,<x1>,<y1>,<c1>,<x2>,<y2>,<c2>,...,<xK>,<yK>,<cK>


In [5]:
import shutil

shutil.copy('runs/detect/train_phase2/weights/best.pt', 'YoloTralaleloTralala.pt')

'YoloTralaleloTralala.pt'

In [9]:
import cv2
import numpy as np
from pathlib import Path
from ultralytics import YOLO
from collections import defaultdict
import yaml

class SurgicalKeypointTracker:
    def __init__(self, model_path):
        """
        Inicializa o tracker com o modelo treinado
        
        Args:
            model_path: Caminho para o modelo treinado (.pt file)
        """
        self.model = YOLO(model_path)
        self.track_history = defaultdict(lambda: [])
        
        # Mapeamento de classes
        self.class_names = {
            0: "needle",
            1: "clamp", 
            2: "curved_scissor",
            3: "straight_scissor"
        }
        
        # Configuração de keypoints por classe
        self.keypoint_configs = {
            0: ['tip', 'base'],  # needle
            1: ['tip1', 'tip2', 'hinge'],  # clamp
            2: ['tip1', 'tip2', 'hinge'],  # curved_scissor
            3: ['tip1', 'tip2', 'hinge']   # straight_scissor
        }
    
    def extract_keypoints_from_bbox(self, bbox, class_id, confidence):
        """
        Extrai keypoints estimados baseados na bounding box
        
        Args:
            bbox: [x1, y1, x2, y2] coordenadas da bounding box
            class_id: ID da classe do instrumento
            confidence: Confiança da detecção
            
        Returns:
            Lista de keypoints no formato [x1, y1, c1, x2, y2, c2, ...]
        """
        x1, y1, x2, y2 = bbox
        width = x2 - x1
        height = y2 - y1
        center_x = x1 + width / 2
        center_y = y1 + height / 2
        
        keypoints = []
        kp_names = self.keypoint_configs.get(class_id, ['center'])
        
        if class_id == 0:  # needle
            # Tip na extremidade superior, base na inferior
            tip_x = center_x
            tip_y = y1 + height * 0.1
            base_x = center_x
            base_y = y2 - height * 0.1
            
            keypoints.extend([tip_x, tip_y, confidence * 0.9])
            keypoints.extend([base_x, base_y, confidence * 0.85])
            
        elif class_id in [1, 2, 3]:  # clamp, curved_scissor, straight_scissor
            # Tip1 e tip2 nas extremidades, hinge no centro
            tip1_x = x1 + width * 0.2
            tip1_y = y1 + height * 0.3
            
            tip2_x = x2 - width * 0.2
            tip2_y = y1 + height * 0.3
            
            hinge_x = center_x
            hinge_y = y2 - height * 0.2
            
            keypoints.extend([tip1_x, tip1_y, confidence * 0.9])
            keypoints.extend([tip2_x, tip2_y, confidence * 0.9])
            keypoints.extend([hinge_x, hinge_y, confidence * 0.85])
        
        return keypoints
    
    def track_video_to_mot_format(self, video_path, output_path=None):
        """
        Realiza tracking em um vídeo e salva no formato MOTChallenge modificado
        
        Args:
            video_path: Caminho para o vídeo de entrada
            output_path: Caminho para salvar o arquivo de tracking
            
        Returns:
            Lista de linhas no formato MOTChallenge modificado
        """
        # Abrir vídeo
        cap = cv2.VideoCapture(str(video_path))
        
        if not cap.isOpened():
            print(f"Erro ao abrir o vídeo: {video_path}")
            return None
        
        # Propriedades do vídeo
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        print(f"Processando vídeo: {video_path}")
        print(f"Total de frames: {total_frames}")
        
        frame_count = 0
        mot_lines = []
        
        try:
            while cap.isOpened():
                success, frame = cap.read()
                
                if not success:
                    break
                
                frame_count += 1
                print(f"Processando frame {frame_count}/{total_frames}", end='\r')
                
                # Realizar tracking com YOLO
                results = self.model.track(frame, persist=True, verbose=False)
                
                # Processar resultados e gerar linhas MOT
                frame_mot_lines = self.process_frame_to_mot(results, frame_count)
                mot_lines.extend(frame_mot_lines)
        
        finally:
            cap.release()
        
        print(f"\nProcessamento concluído!")
        
        # Salvar arquivo se especificado
        if output_path:
            with open(output_path, 'w') as f:
                for line in mot_lines:
                    f.write(line + '\n')
            print(f"Arquivo MOT salvo em: {output_path}")
        
        return mot_lines
    
    def process_frame_to_mot(self, results, frame_id):
        """
        Processa os resultados de um frame e converte para formato MOTChallenge modificado
        
        Args:
            results: Resultados do YOLO tracking
            frame_id: ID do frame atual
            
        Returns:
            Lista de strings no formato MOTChallenge modificado
        """
        mot_lines = []
        
        if results[0].boxes is not None and results[0].boxes.id is not None:
            # Extrair informações das detecções
            boxes = results[0].boxes.xyxy.cpu().numpy()  # [x1, y1, x2, y2]
            track_ids = results[0].boxes.id.int().cpu().tolist()
            confidences = results[0].boxes.conf.cpu().numpy()
            classes = results[0].boxes.cls.int().cpu().tolist()
            
            # Processar cada detecção
            for bbox, track_id, conf, cls in zip(boxes, track_ids, confidences, classes):
                # Extrair keypoints da bounding box
                keypoints = self.extract_keypoints_from_bbox(bbox, cls, conf)
                
                # Criar linha no formato MOTChallenge modificado
                # <frame>, <id>, <x1>, <y1>, <c1>, <x2>, <y2>, <c2>, ..., <xK>, <yK>, <cK>
                mot_line = f"{frame_id},{track_id}"
                
                for i in range(0, len(keypoints), 3):
                    x, y, c = keypoints[i:i+3]
                    mot_line += f",{x:.2f},{y:.2f},{c:.3f}"
                
                mot_lines.append(mot_line)
        
        return mot_lines
    
    def track_single_frame(self, frame, frame_id):
        """
        Processa um único frame e retorna resultados MOT
        
        Args:
            frame: Frame de vídeo (numpy array)
            frame_id: ID do frame
            
        Returns:
            Lista de strings no formato MOTChallenge modificado
        """
        results = self.model.track(frame, persist=True, verbose=False)
        return self.process_frame_to_mot(results, frame_id)
    
    def get_keypoint_info(self):
        """
        Retorna informações sobre os keypoints configurados
        
        Returns:
            Dicionário com informações dos keypoints por classe
        """
        info = {}
        for class_id, class_name in self.class_names.items():
            keypoint_names = self.keypoint_configs.get(class_id, [])
            info[class_name] = {
                'class_id': class_id,
                'keypoints': keypoint_names,
                'num_keypoints': len(keypoint_names)
            }
        return info

def main():
    
    model_path = "YoloTralaleloTralala.pt"
    
    # Verificar se o modelo existe
    if not Path(model_path).exists():
        print(f"Modelo não encontrado em: {model_path}")
        print("Certifique-se de que o treino foi concluído e o modelo foi salvo.")
        return
    
    # Inicializar tracker
    tracker = SurgicalKeypointTracker(model_path)
    
    # Mostrar informações dos keypoints
    print("Configuração de keypoints:")
    keypoint_info = tracker.get_keypoint_info()
    for class_name, info in keypoint_info.items():
        print(f"  {class_name}: {info['keypoints']} ({info['num_keypoints']} keypoints)")
    
    video_path = "../OSS_dataset/Train/videos/A31H.mp4"
    
    # Verificar se o vídeo existe
    if not Path(video_path).exists():
        print(f"Vídeo não encontrado em: {video_path}")
        print("Por favor, forneça o caminho correto para um vídeo cirúrgico.")
        return
    
    # Realizar tracking e salvar no formato MOTChallenge modificado
    output_path = "surgical_tracking_mot_keypoints.txt"
    mot_lines = tracker.track_video_to_mot_format(
        video_path=video_path,
        output_path=output_path
    )
    
    # Mostrar algumas linhas de exemplo
    if mot_lines:
        print(f"\nExemplo de saída (primeiras 5 linhas):")
        for i, line in enumerate(mot_lines[:5]):
            print(f"  {line}")
        
        print(f"\nTotal de detecções: {len(mot_lines)}")
        print(f"Formato: <frame>,<id>,<x1>,<y1>,<c1>,<x2>,<y2>,<c2>,...,<xK>,<yK>,<cK>")

# Exemplo de uso alternativo para processamento frame por frame
def process_video_frame_by_frame(model_path, video_path):
    """
    Exemplo de processamento frame por frame
    """
    tracker = SurgicalKeypointTracker(model_path)
    cap = cv2.VideoCapture(str(video_path))
    
    frame_id = 1
    all_mot_lines = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Processar frame individual
        frame_mot_lines = tracker.track_single_frame(frame, frame_id)
        all_mot_lines.extend(frame_mot_lines)
        
        # Mostrar progresso
        if frame_id % 10 == 0:
            print(f"Processado frame {frame_id}")
        
        frame_id += 1
    
    cap.release()
    return all_mot_lines

if __name__ == "__main__":
    main()


Configuração de keypoints:
  needle: ['tip', 'base'] (2 keypoints)
  clamp: ['tip1', 'tip2', 'hinge'] (3 keypoints)
  curved_scissor: ['tip1', 'tip2', 'hinge'] (3 keypoints)
  straight_scissor: ['tip1', 'tip2', 'hinge'] (3 keypoints)
Processando vídeo: ../OSS_dataset/Train/videos/A31H.mp4
Total de frames: 9075
Processando frame 9075/9075
Processamento concluído!
Arquivo MOT salvo em: surgical_tracking_mot_keypoints.txt

Exemplo de saída (primeiras 5 linhas):
  1,1,893.24,484.78,0.427,1261.96,484.78,0.427,1077.60,643.87,0.403
  2,1,893.06,484.23,0.252,1270.55,484.23,0.252,1081.81,643.83,0.238
  3,1,1082.59,436.23,0.556,1082.59,662.42,0.525
  4,1,1066.67,447.67,0.350,1066.67,654.89,0.331
  5,1,1079.91,452.60,0.458,1079.91,659.50,0.433

Total de detecções: 29993
Formato: <frame>,<id>,<x1>,<y1>,<c1>,<x2>,<y2>,<c2>,...,<xK>,<yK>,<cK>
